In [33]:
# f = (f' - B - d * t) / med (f' - b - d * t)
# D = D' - B
# B = B' + Berr

In [34]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.optimize import curve_fit

In [35]:
# # parameters
# # gain electrons per adu
# g = 3.6
# # dimension 
# row = 2048
# column = 2048
# # bias levels 
# # real bias level count per pixel 
# bias_real_level = 300
# # the systematic readout error on the bias
# readnoise = 5
# # assumed bias level 
# # float up to factor 
# factor = 2 
# bias_meas_array = np.linspace(bias_real_level/factor, bias_real_level*factor)
# # bias_meas_array = [bias_real_level]
# # dark
# # dark current elec per sec
# d = 0.05
# # time dark exposure
# td = 60
# # flat counts per pixel 
# f = 10000
# # time flat exposure 
# tf_array = np.linspace(3, 120)

In [36]:
# para_dict = {
#     'bias_level': 300,
#     'row': 2048,
#     'column': 2048,
#     'g': 3.6,
#     'readnoise': 5,
#     'd': 0.05,
#     'td': 60,
#     'f': 10000,
#     'tf_array': np.linspace(3, 120)   
# }

In [37]:
# function making a bias based on a given bias input
# normal distributed 
# output: 2d bias level in counts per pixel 
# def makeBias(bias_level,size=(row,column),readnoise=5,debug=False):
#     bias = np.random.normal(bias_level, readnoise, size)
#     return bias

In [38]:
# def makeDarkCounts(td=60,d=0.05,size=(row,column),debug=False,g=3.6):
#     dark_counts = np.random.poisson(d*td, size) / g 
#     return dark_counts

In [39]:
# # dark function based on a given bias input
# # possion distributed
# # output: 2d dark current in counts per sec per pixel 
# def makeDark(bias_level,td=60,d=0.05,size=(row,column),debug=False,g=3.6):
#     dark = makeDarkCounts(td=td,d=d,size=size,debug=debug,g=g) + \
#                               makeBias(bias_level,size=size,debug=debug)
#     return dark    

In [40]:
# # flat function 
# # possion distributed 
# # output: 2d raw flat level in counts per pixel 
# def makeRawFlat(bias_level,debug=False,g=3.6,
#                 f=15000,size=(row,column)):
#     flat = makeBias(bias_level) + np.random.poisson(f, size)
#     return flat

In [41]:
# # calibrated flat function 
# # output: 2d calibrated flat level in counts per pixel 
# def makeCaliFlat(bias, dark, raw_flat, tf=3, td=60, debug = False):
#     caliFlat = (raw_flat - dark / td * tf - bias) / np.median(raw_flat - dark / td * tf - bias)
#     return caliFlat 

In [42]:
# import numpy as np
# import matplotlib.pyplot as plt

# def generate_custom_gaussian_circle(resolution, center_value, sigma, edge_value):
#     center_x, center_y = resolution / 2, resolution / 2
#     grid = np.zeros((resolution, resolution))
    
#     for x in range(resolution):
#         for y in range(resolution):
#             distance = np.sqrt((x - center_x)**2 + (y - center_y)**2)
#             # Calculate the Gaussian-like value with a custom mean
#             value = (np.random.normal(300, 5) + np.random.poisson(center_value)) * np.exp(-0.5 * (distance / sigma)**2) 
            
#             # Clamp values to be within [edge_value, center_value]
#             # value = max(edge_value, min(center_value, value))
            
#             # Assign the value to the grid
#             grid[x, y] = value
    
#     return grid

# resolution = 2048
# center_value = 15000  # Adjust this value for the center peak
# edge_value = 0   # Adjust this value for the edge
# sigma = 4000          # Adjust this value to control the spread of values

# custom_gaussian_circle = generate_custom_gaussian_circle(resolution, center_value, sigma, edge_value)
# display(np.mean(custom_gaussian_circle))

# plt.imshow(custom_gaussian_circle, cmap='viridis')
# plt.colorbar()
# plt.show()



In [43]:
# def systematic_deviation(bias_level, time = 1, tf_array = tf_array, debug = False):
#     # see the error with same bias 
#     percent_change_in_flat_array = []
#     percent_change_in_flat_array_std = []
#     tf_plot_array = []
#     for tf in tf_array:
#         percent_change_in_flat_array_storage = []
#         for i in range(time):
#             bias_real_1 = makeBias(bias_level)
#             dark_real_1 = makeDark(bias_level)
#             flat_raw_1 = makeRawFlat(bias_level, tf)
#             flat_real_1 = makeCaliFlat(bias_real_1, dark_real_1, flat_raw_1, tf)
#             bias_real = makeBias(bias_level)
#             dark_real = makeDark(bias_level)
#             flat_raw = makeRawFlat(bias_level, tf)
#             flat_real = makeCaliFlat(bias_real, dark_real, flat_raw, tf)
#             deltaf = np.abs(flat_real - flat_real_1)
#             # flat percent change in a mean form 
#             percent_change_in_flat = np.mean(deltaf / flat_real_1 * 100)
#             percent_change_in_flat_array_storage.append(percent_change_in_flat)
#         mean = np.mean(percent_change_in_flat_array_storage)
#         std = np.std(percent_change_in_flat_array_storage)
#         percent_change_in_flat_array.append(mean)
#         percent_change_in_flat_array_std.append(std)
#         tf_plot_array.append(tf)
#     plt.scatter(tf_plot_array, percent_change_in_flat_array)
#     plt.errorbar(tf_plot_array, percent_change_in_flat_array,
#                 yerr = percent_change_in_flat_array_std,
#                 fmt ='o')
#     plt.xlabel('TF changes')
#     plt.ylabel('Flat changes perc')
#     plt.show()
    

In [44]:
#----------------------------------------------------------------------
# Overall concept
#----------------------------------------------------------------------

# f = (f' - B - d * t) / med (f' - b - d * t)
# D = D' - B
# B = B' + Berr
#

#----------------------------------------------------------------------

# modelues using versions on bellerophon
import numpy as np
import matplotlib.pyplot as plt
# from quick_image import *
from scipy.optimize import curve_fit

#----------------------------------------------------------------------

# default parameters
# ccd dimension (pixels)
row = 2048
column = 2048
# gain of the camera (electron per adu)
g = 0.95
# bias levels 
# real bias level (count per pixel)
bias_real_level = 300
# assumed bias level 
# float up to factor 
factor = 2 
bias_meas_array = np.linspace(bias_real_level/factor, bias_real_level*factor)
# bias read out noise (counts per pixel)
readnoise = 5
# dark 
# dark current (electrons per second per pixel)
d = 0.05
# dark exposure time (sec)
td = 60
# flat 
# flat (counts per pixel)
f = 15000
# time flat exposure (sec)
tf_array = np.linspace(3, 120)


#----------------------------------------------------------------------
# bias, dark, and flat simulation frame


#----------------------------------------------------------------------
# function making a bias based on a given bias input 
# default para
# size, readnoise, and debug util
# input, bias_level (counts per pixel)
# bias level normally distributed 
# output: 2d bias level in counts per pixel
def makeBias(bias_level,size=(row,column),readnoise=readnoise,debug=False):
    bias = np.random.normal(bias_level, readnoise, size)
    if debug:
        display_image(bias,siglo=0.5,sighi=0.5,fignum=1)
        plt.colorbar()
    return bias

#----------------------------------------------------------------------
# fucntion making the raw dark counts 
# default para
# size, debug until, gain (g), time for dark exposure (td), and dark current (d)
# possion distributed
# output: 2d dark current in counts per pixel 
def makeDarkCounts(td=td,d=d,size=(row,column),debug=False,g=g):
    dark_counts = np.random.poisson(d*td, size) / g 
    if debug:
        display_image(dark_counts,siglo=0.5,sighi=0.5,fignum=2)
        plt.colorbar()
    return dark_counts

# function making the dark counts with the bias level based on a given bias input 
# default para
# time for dark exposure (td), dark curent (d), size, debug until, gain (g)
# input bias level (counts per pixel)
# calls the makeDarkCountrs function 
# output 2d true dark current in counts per pixel 
def makeDark(bias_level,td=td,d=d,size=(row,column),debug=False,g=g):
    dark = makeDarkCounts(td=td,d=d,size=size,debug=debug,g=g) + \
                              makeBias(bias_level,size=size,debug=debug)
    if debug:
        display_image(dark,siglo=0.5,sighi=0.5,fignum=3)
        plt.colorbar()
    return dark     


#----------------------------------------------------------------------
# function making the flat frame counts given a bias level input 
# default para
# flat counts (f), debug util, gain (g), size 
# input, bias level (counts per pixel)
# possion distributed 
# output: 2d raw flat level in counts per pixel 
def makeRawFlat(bias_level,debug=False,g=g,
                f=f,size=(row,column)):
    flat = makeBias(bias_level) + np.random.poisson(f, size)
    if debug:
        display_image(flat,siglo=0.5,sighi=0.5,fignum=4)
        plt.colorbar()
    return flat


# flat function with Gaussian circle
# added default para, sigma = determines how quick it is going to drop from the center
# To do: decrease run time
def makeRawGaussianFlat(bias_level,readnoise = readnoise, debug=False,g=g,
                f=f,size=(row,column), sigma = 2048):
    center_x, center_y = size[0] / 2, size[1] / 2
    flat = np.zeros(size)
    for x in range (size[0]):
        for y in range (size[1]):
            distance = np.sqrt((x - center_x)**2 + (y - center_y)**2)
            value = (np.random.normal(bias_level, readnoise) + np.random.poisson(f)) * np.exp(-0.5 * (distance / sigma)**2) 
            flat[x, y] = value
            
    if debug:
        display_image(flat,siglo=0.5,sighi=1,fignum=4)
        plt.colorbar()
        plt.imshow(flat, cmap='viridis')
        plt.colorbar()
        plt.show()

    return flat

#----------------------------------------------------------------------
# calibrated flat function 
# output: 2d calibrated flat level in counts per pixel 
# be looked at gaiz
def makeCaliFlatAll(bias_level, bias_real_level = bias_real_level, d = d, f = f, tf=3, td=td, debug = False, size = (row, column), g = g):
    bias = makeBias(bias_level, debug = debug, size = size) 
    dark = makeDark(bias_level, d = d, td = td, size = size, debug = debug, g = g)
    raw_flat = makeRawGaussianFlat(bias_level, g = g, debug = debug, f = f, size = size)
    caliFlat = (raw_flat - dark / td * tf - bias) / np.median(raw_flat - dark / td * tf - bias)
    if debug:
        display_image(caliFlat,siglo=0.5,sighi=0.5,fignum=5)
        plt.colorbar()
    return caliFlat 

# calibrated flat function 
# output: 2d calibrated flat level in counts per pixel 
def makeCaliFlat(bias, dark, raw_flat, tf=3, td=60, debug = False):
    caliFlat = (raw_flat - dark / td * tf - bias) / np.median(raw_flat - dark / td * tf - bias)
    return caliFlat 


In [45]:
def func(xy, a, b, c, d, e, f):
    x, y = xy
    return a + b*x + c*y + d*x**2 + e*y**2 + f*x*y

In [46]:
def propagation(bias_real_level = bias_real_level, bias_meas_array = bias_meas_array, tf_array = tf_array, debug = False):
    # prop storage array
    percent_change_in_bias_array = []
    percent_change_in_flat_array = []
    tf_plot_array = []

    # making the real bias, dark, and flat
    bias_real = makeBias(bias_real_level)
    dark_real = makeDark(bias_real_level)
    flat_raw = makeRawGaussianFlat(bias_real_level)
    flat_real_dict = {}
    for tf in tf_array:
        flat_real = makeCaliFlat(bias_real, dark_real, flat_raw, tf)
        flat_real_dict[tf] = flat_real
        
    if debug:
        display(flat_real_dict[tf_array[1]])
        
    # prop 
    for b in bias_meas_array:
        # bias displacement
        deltab = b - bias_real_level
        # bias percent change mean
        percent_change_in_bias = (b - bias_real) / bias_real_level
        for tf in tf_array:
            # real calibrated flat in a 2d array 
            flat_real = flat_real_dict[tf]
            # measured flat in a 2d array
            flat_meas = makeCaliFlatAll(b)
            # flat displacement in a 2d 
            deltaf = np.abs(flat_meas - flat_real)
            # flat percent change in a mean form 
            percent_change_in_flat = np.mean(deltaf / flat_real * 100)
            percent_change_in_bias_array.append(percent_change_in_bias)
            percent_change_in_flat_array.append(percent_change_in_flat)
            tf_plot_array.append(tf)
            
    if debug:
        ax = plt.axes(projection='3d')
        # Adding data to our axes, 's=40' is to increase point size by 40
        ax.scatter3D(percent_change_in_bias_array, tf_plot_array, percent_change_in_flat_array, s=1) 
        # Labelling your axes
        ax.set_xlabel('Percent change in Bias')
        ax.set_ylabel('Time Flat Exposure')
        ax.set_zlabel('Percent change in Flat')
        
    
    # Perform curve fitting
    popt, pcov = curve_fit(func, (percent_change_in_bias_array, tf_plot_array), percent_change_in_flat_array)
    
    if debug: 
            
        X, Y = np.meshgrid(percent_change_in_bias_array, tf_plot_array)
        pred_flat_change = func((X, Y), *popt)
        
        ax = plt.axes(projection='3d')
        # Adding data to our axes, 's=40' is to increase point size by 40
        ax.scatter3D(percent_change_in_bias_array, tf_plot_array, percent_change_in_flat_array, s=1) 
        ax.plot_surface(X, Y, pred_flat_change, color='red', alpha=0.5)
        # Labelling your axes
        ax.set_xlabel('Percent change in Bias')
        ax.set_ylabel('Time Flat Exposure')
        ax.set_zlabel('Percent change in Flat')
        
    return popt, pcov
            
    

In [47]:
popt, pcov = propagation(debug = True)

KeyboardInterrupt: 